In [185]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
import tensorflow as tf

In [187]:
import os
import tensorflow_datasets as tfds
from sklearn.utils import shuffle

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# strategy = tf.distribute.TPUStrategy(resolver)

In [188]:
# !pip install -q git+https://github.com/tensorflow/docs

In [189]:
# from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import gc
from sklearn.metrics import accuracy_score
import time
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [190]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [191]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [192]:
def changePath(row):
  path=row["Path"]
  path=path.replace("Transformer_1","dcsass")
  return path

In [193]:
csv=pd.read_csv("/content/drive/MyDrive/tf_transformer_4/data.csv")
csv=csv[csv['exists']==True]
csv=csv[csv['classLabel']<=15]

In [194]:
import pandas as pd
from sklearn.model_selection import train_test_split
trainCsv=pd.DataFrame()
valCsv=pd.DataFrame()
testCsv=pd.DataFrame()
n_of_each_class=400
csv=csv[csv['exists']==True]
csv=csv[csv['classLabel']<=15]
csv["Path"]=csv.apply(lambda row:changePath(row),axis=1)

In [195]:
# csv[csv["label"]==0]

In [196]:
# n_classes=len((csv['classLabel'].unique()))
# print(sorted((csv['classLabel'].unique())))
# for Class in sorted((csv['classLabel'].unique())):
#   t1=len(csv[csv['classLabel']==Class])
#   t,v=train_test_split(csv[csv['classLabel']==Class].sample(min(t1,n_of_each_class)),test_size=0.30)
#   v1,tes=train_test_split(v[v['classLabel']==Class],test_size=1/3)
#   trainCsv=pd.concat((trainCsv,t))
#   testCsv=pd.concat((testCsv,tes))
#   valCsv=pd.concat((valCsv,v1))

In [197]:
option="classLabel"
# option="label"
n_classes=len((csv[option].unique()))
print(sorted((csv['classLabel'].unique())))
for Class in sorted((csv['class'].unique())):
  # t1=len(csv[csv['classLabel']==Class])
  chunk0=csv[(csv['class']==Class)&(csv['label']==0.0)]
  t0=len(chunk0)
  chunk1=csv[(csv['class']==Class)&(csv['label']==1.0)]
  t1=len(chunk1)
  chunk=pd.concat([chunk0.sample(min(t1,min(t0,n_of_each_class//2))),chunk1.sample(min(t1,n_of_each_class//2))])
  # chunk=pd.concat([chunk0.sample(min(0,min(t0,n_of_each_class//2))),chunk1.sample(min(t1,n_of_each_class//2))])
  t,v=train_test_split(chunk,test_size=0.30)
  v1,tes=train_test_split(v,test_size=1/3)
  trainCsv=pd.concat((trainCsv,t))
  testCsv=pd.concat((testCsv,tes))
  valCsv=pd.concat((valCsv,v1))
class_count_dict=dict()
max_class=0
for Class in trainCsv[option].unique():
  t=len(trainCsv[trainCsv[option]==Class])
  max_class=max(max_class,t)
for Class in trainCsv[option].unique():
  t=len(trainCsv[trainCsv[option]==Class])
  class_count_dict[Class]=max_class/t
print(class_count_dict)
trainCsv = shuffle(trainCsv)
testCsv = shuffle(testCsv)
valCsv = shuffle(valCsv)
print(len(trainCsv))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0]
{0.0: 1.0, 1.0: 12.511111111111111, 3.0: 12.511111111111111, 2.0: 11.97872340425532, 5.0: 11.811188811188812, 11.0: 12.151079136690647, 13.0: 14.313559322033898, 10.0: 28.15, 8.0: 12.23913043478261, 9.0: 12.699248120300751, 6.0: 11.648275862068965, 4.0: 14.81578947368421, 12.0: 12.419117647058824, 7.0: 12.795454545454545}
3358


In [198]:
for Class in csv["class"].unique():
  print(len(trainCsv[(trainCsv["label"]==0)&(trainCsv["class"]==Class)]))
  print(len(trainCsv[(trainCsv["label"]==1)&(trainCsv["class"]==Class)]))
  print("-------")

145
135
-------
139
141
-------
145
135
-------
103
114
-------
137
143
-------
135
145
-------
148
132
-------
142
138
-------
147
133
-------
60
60
-------
141
139
-------
144
136
-------
103
118
-------


In [199]:
print(len(trainCsv[trainCsv["label"]==0]))
print(len(trainCsv[trainCsv["label"]==1]))

1689
1669


In [200]:
frameSize=(128,128)
nFrames=45
# defFrame=(300,300)
batch_size=32#8
vals_batch_size=32#8
params = {'batch_size': batch_size,
          'frameSize': frameSize,
          # 'frameSize': defFrame,
          'nFrames': nFrames,
          'nclasses': n_classes,
          'oneHotEncode': True,
          'shuffle': True, 
          'normalize': False,
          'std':False}
val_params={'batch_size': vals_batch_size,
          'frameSize': frameSize,
          # 'frameSize': defFrame,
          'nFrames': nFrames,
          'nclasses': n_classes,
          'oneHotEncode': True,
          'shuffle': True, 
          'normalize': False,
          'std':False}

In [201]:
!pip install --upgrade ipyparallel
!ipcluster start -n 24

2022-02-07 16:12:19.010 [IPClusterStart] Using existing profile dir: '/root/.ipython/profile_default'
2022-02-07 16:12:19.643 [IPClusterStart] CRITICAL | Cluster is already running at /root/.ipython/profile_default/security/cluster-.json. use `ipcluster stop` to stop the cluster.


In [202]:
trainCsv["Path"].iloc[[0]].item()

'/content/drive/MyDrive/dcsass/DCSASS Dataset/Explosion/Explosion052_x264.mp4/Explosion052_x264_17.mp4'

In [203]:
# from ipyparallel import Client
# with tf.device("/device:GPU:0"): 
# # with strategy.scope(): 
#   cli = Client()
#   dview = cli[:]
#   @dview.parallel(block=True)
#   def loadData(i,index,listPaths,labels,nFrames,frameSize):
#     import numpy as np
#     import cv2,gc
#     path = listPaths[index]
#     try:
#       vidcap=cv2.VideoCapture(path)
#       totalFrames=int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
#       # print(totalFrames)
#       if totalFrames>=nFrames:
#         focusFrames=np.linspace(0,totalFrames-1,nFrames).astype(int)
#         # assert len(focusFrames) == len(set(focusFrames))
#         success=True
#         imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
#         i1=0
#         countPush=0
#         while success==True:
#           success,image = vidcap.read()
#           if i1 in focusFrames:
#             img=cv2.resize(image,frameSize)
#             imgsBundle[countPush]=img
#             # imgsBundle[countPush]=image
#             countPush+=1
#           if countPush==nFrames-1:
#             break
#           i1+=1
#         gc.enable()
#         gc.collect()
#         labels1=labels[index]
#       else:
#         # pass
#         imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
#         i1=0
#         countPush=0
#         while success==True:
#           success,image = vidcap.read()
#           img=cv2.resize(image,frameSize)
#           imgsBundle[countPush]=img
#           # imgsBundle[countPush]=image
#           countPush+=1
#           i1+=1
#         gc.enable()
#         gc.collect()
#         labels1=labels[index]
#       return imgsBundle,labels1
#     except:
#       imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
#       labels1=0
#       gc.enable()
#       gc.collect()
#       print("Weird data")
#       return imgsBundle,labels1

In [204]:
from ipyparallel import Client
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  cli = Client()
  dview = cli[:]
  @dview.parallel(block=True)
  def loadData(i,index,listPaths,labels,nFrames,frameSize):
    import numpy as np
    import cv2,gc
    path = listPaths[index]
    try:
      vidcap=cv2.VideoCapture(path)
      totalFrames=int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
      print(totalFrames)
      if totalFrames>=nFrames:
        # print("Case 1")
        focusFrames=np.linspace(0,totalFrames-1,nFrames).astype(int)
        # assert len(focusFrames) == len(set(focusFrames))
        success=True
        imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
        i1=0
        countPush=0
        while success==True:
          success,image = vidcap.read()
          if success==False:
            break
          if i1 in focusFrames:
            img=cv2.resize(image,frameSize)
            imgsBundle[countPush]=img
            # imgsBundle[countPush]=image
            countPush+=1
          if countPush==nFrames-1:
            break
          i1+=1
        gc.enable()
        gc.collect()
        labels1=labels[index]
      else:
        # pass
        print("Case 2")
        imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
        i1=0
        countPush=0
        success=True
        while success==True:
          success,image = vidcap.read()
          if success==False:
            break
          # print("Got here: ",countPush)
          img=cv2.resize(image,frameSize)
          imgsBundle[countPush]=img
          # imgsBundle[countPush]=image
          countPush+=1
          i1+=1
        gc.enable()
        gc.collect()
        labels1=labels[index]
      return imgsBundle,labels1
    except:
      imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
      labels1=0
      gc.enable()
      gc.collect()
      print("Weird data")
      return imgsBundle,labels1

In [205]:
# for i,row in trainCsv.iterrows():
#   if os.path.exists(row["Path"])==False:
#     print(row["Path"])

In [206]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  # cli = Client()
  # dview = cli[:]
  # @dview.parallel(block=True)
  def loadDataTest(path,nFrames,frameSize):
      import numpy as np
      import cv2,gc
      fine=True
      try:
        vidcap=cv2.VideoCapture(path)
        totalFrames=int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(totalFrames)
        fine=True
        if totalFrames>=nFrames:
          print("Case 1")
          focusFrames=np.linspace(0,totalFrames-1,nFrames).astype(int)
          # assert len(focusFrames) == len(set(focusFrames))
          success=True
          imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
          i1=0
          countPush=0
          while success==True:
            success,image = vidcap.read()
            if success==False:
              break
            if i1 in focusFrames:
              img=cv2.resize(image,frameSize)
              imgsBundle[countPush]=img
              # imgsBundle[countPush]=image
              countPush+=1
            if countPush==nFrames-1:
              break
            i1+=1
          gc.enable()
          gc.collect()
        else:
          # pass
          print("Case 2")
          imgsBundle=np.zeros((nFrames,frameSize[0],frameSize[1],3))
          i1=0
          countPush=0
          success=True
          while success==True:
            success,image = vidcap.read()
            if success==False:
              break
            # print("Got here: ",countPush)
            img=cv2.resize(image,frameSize)
            imgsBundle[countPush]=img
            # imgsBundle[countPush]=image
            countPush+=1
            i1+=1
          gc.enable()
          gc.collect()

      except:
        fine=False
        gc.enable()
        gc.collect()
        print("Weird data")
      print("-----------------")
      return fine

In [207]:
# fails=0
# for i,row in trainCsv.iterrows():
#   if loadDataTest(row["Path"],nFrames,frameSize)==False:
#     fails+=1
# print(fails)

In [208]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 

  class Dataset(keras.utils.Sequence):
    'Characterizes a dataset for PyTorch'
    def __init__(self, listPaths, labels,batch_size=10,nFrames=30,frameSize=(300,300),nclasses=2,shuffle=True,oneHotEncode=False,normalize=True,std=False):
          'Initialization'
          self.labels = labels
          self.listPaths = listPaths
          self.nFrames = nFrames
          self.frameSize = frameSize
          self.oneHotEncode = oneHotEncode
          self.nclasses=nclasses
          self.shuffle=shuffle
          self.std=std
          self.normalize=normalize
          self.batch_size = batch_size
          self.on_epoch_start()
          # print('Data len',len(self.listPaths))

    def __len__(self):
      'Denotes the total number of samples'
      #for example lets say we have 2000 videos for training and batch size is 40, then number of samples=2000/40=50
      return int(np.floor(len(self.labels) / self.batch_size))

    def __getitem__(self,index):
      #since number of samples is 50 python will call this iterable using the __getitem__ function and index argument will go from 0 to 49
      indexes = self.indexes[(index*self.batch_size):((index+1)*self.batch_size)]
      Xx,Yx=self.__data_generation(indexes)
      Xx=tf.convert_to_tensor(Xx)
      Yx=tf.convert_to_tensor(Yx)
      return Xx,Yx
      
    def on_epoch_start(self):
      'Updates indexes after each epoch'
      self.indexes = np.arange(len(self.labels))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)
    def __data_generation(self, indexes):
      'Generates one sample of data'
      # data=loadData.map([{i,index,self.listPaths,self.labels,self.nFrames,self.frameSize} for i,index in enumerate(indexes)])
      data=loadData.map(range(len(indexes)),indexes,[self.listPaths]*len(indexes),[self.labels]*len(indexes),[self.nFrames]*len(indexes),[self.frameSize]*len(indexes))
      # Y=np.asarray(data)[:,1]
      Y=(np.asarray(data)[:,1]).astype(int)
      X=np.stack(np.asarray(data)[:,0],axis=0)
      if self.oneHotEncode==True:
        # print(self.nclasses)
        Y=keras.utils.to_categorical(Y,self.nclasses)
      if self.std==True and np.std(X)!=0:
        X=(X-np.mean(X))/np.std(X)
      if self.normalize==True:
        X=X/255
      # print("Shape of output by data generator: ",np.shape(X),' ',np.shape(Y))
      return X,Y

In [209]:
# trainCsv

In [210]:
# with strategy.scope(): 
with tf.device("/device:GPU:0"): 

  training_generator = Dataset(listPaths=trainCsv['Path'].values, labels=trainCsv[option].values, **params)
  val_generator = Dataset(listPaths=valCsv['Path'].values,labels= valCsv[option].values, **val_params)

In [211]:
# from matplotlib import pyplot as plt
# from google.colab.patches import cv2_imshow
# failCount=0
# total=0
# consecutiveSuccess=0
# for data_ in training_generator:
#   # plt.imshow(data_[0][0][0], interpolation='nearest')
#   total+=1
#   if np.count_nonzero(data_[0])==0:
#     failCount+=1
#     consecutiveSuccess=0
#   else:
#     consecutiveSuccess+=1
#   print(failCount," ",total)

In [212]:
# with strategy.scope(): 
with tf.device("/device:GPU:0"): 
  class PositionalEmbedding(layers.Layer):
      def __init__(self, sequence_length, output_dim, **kwargs):
          super().__init__(**kwargs)
          self.position_embeddings = layers.Embedding(
              input_dim=sequence_length, output_dim=output_dim
          )
          self.sequence_length = sequence_length
          self.output_dim = output_dim

      def call(self, inputs):
          # The inputs are of shape: `(batch_size, frames, num_features)`
          length = tf.shape(inputs)[1]
          positions = tf.range(start=0, limit=length, delta=1)
          embedded_positions = self.position_embeddings(positions)
          return inputs + embedded_positions

      def compute_mask(self, inputs, mask=None):
          mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
          return mask

In [213]:
with tf.device("/device:GPU:0"): 
  class TFPositionalEncoding1D(tf.keras.layers.Layer):
    def __init__(self, channels: int, dtype=tf.float32):
        """
        Args:
            channels int: The last dimension of the tensor you want to apply pos emb to.
        Keyword Args:
            dtype: output type of the encodings. Default is "tf.float32".
        """
        super(TFPositionalEncoding1D, self).__init__()

        self.channels = int(np.ceil(channels / 2) * 2)
        self.inv_freq = np.float32(
            1
            / np.power(
                10000, np.arange(0, self.channels, 2) / np.float32(self.channels)
            )
        )

    @tf.function
    def call(self, inputs):
        """
        :param tensor: A 3d tensor of size (batch_size, x, ch)
        :return: Positional Encoding Matrix of size (batch_size, x, ch)
        """
        if len(inputs.shape) != 3:
            raise RuntimeError("The input tensor has to be 3d!")
        _, x, org_channels = inputs.shape

        dtype = self.inv_freq.dtype
        pos_x = tf.range(x, dtype=dtype)
        sin_inp_x = tf.einsum("i,j->ij", pos_x, self.inv_freq)
        emb = tf.expand_dims(tf.concat((tf.sin(sin_inp_x), tf.cos(sin_inp_x)), -1), 0)
        emb = emb[0]  # A bit of a hack
        return tf.repeat(emb[None, :, :org_channels], tf.shape(inputs)[0], axis=0)

In [214]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.5):
      # Normalization and Attention
      # x = layers.LayerNormalization(epsilon=1e-6)(inputs)
      x = layers.MultiHeadAttention(
          key_dim=head_size, 
          num_heads=num_heads, 
          dropout=dropout,
          # kernel_regularizer=tf.keras.regularizers.l1(0.02),
          # activity_regularizer=tf.keras.regularizers.l1(0.01)
      )(inputs, inputs)
      x = layers.Dropout(dropout)(x)
      res = x + inputs

      # Feed Forward Part
      # x = layers.LayerNormalization(epsilon=1e-6)(res)
      x = layers.Conv1D(filters=ff_dim, kernel_size=1,activation=tf.nn.gelu)(res)
      # x = layers.Conv1D(filters=20, kernel_size=1, activation="relu")(x)
      # x = layers.Conv1D(filters=20, kernel_size=1, activation="relu")(x)
      x = layers.Dropout(dropout)(x)
      x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
      return x + res

In [215]:
with tf.device("/device:GPU:0"): 
  class TransformerEncoder(layers.Layer):
      def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
          super().__init__(**kwargs)
          self.embed_dim = embed_dim
          self.dense_dim = dense_dim
          self.num_heads = num_heads
          self.attention = layers.MultiHeadAttention(
              num_heads=num_heads, key_dim=embed_dim, dropout=0.3
          )
          self.dense_proj = keras.Sequential(
              [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim, activation=tf.nn.gelu),]
          )
          # self.layernorm_1 = layers.LayerNormalization()
          self.batchnorm_1 = layers.BatchNormalization()
          # self.layernorm_2 = layers.LayerNormalization()
          self.batchnorm_2 = layers.BatchNormalization()

      def call(self, inputs, mask=None):
          if mask is not None:
              mask = mask[:, tf.newaxis, :]
          inputs=self.batchnorm_1(inputs)
          attention_output = self.attention(inputs, inputs, attention_mask=mask)
          # proj_input = self.batchnorm_2(inputs + attention_output)
          proj_input=(inputs + attention_output)
          proj_output = self.dense_proj(proj_input)
          # return self.batchnorm_2(proj_input + proj_output)
          # return(inputs + attention_output)
          # return proj_input
          return proj_output+proj_input
          # return attention_output

In [216]:
# with strategy.scope(): 
with tf.device("/device:GPU:0"): 
  def mainModel(seqLen=nFrames,embedDim=2048,numHeads=32,ff_dim=150,frameSize=frameSize,classes=2,dense_dim2=8,nTr=6,trDr=0.2,trDr2=0.2):
    # print("INC model")
    # inputs = keras.Input(shape=(seqLen, frameSize[0], frameSize[1], 3))
    inputs = keras.Input(shape=(seqLen, *(frameSize), 3))
    print(inputs.shape)
    # x=layers.TimeDistributed(layers.CenterCrop(*(frameSize)))(inputs)
    # print(x.shape)
    # x=layers.Lambda(lambda x:x/255)(inputs)
    # x=keras.applications.inception_v3.preprocess_input(
    #     inputs
    # )
    # x=keras.applications.inception_v3.preprocess_input(
    #     inputs
    # )
    x=keras.applications.densenet.preprocess_input(inputs)
    # inc=InceptionV3(include_top=False, pooling='avg',input_shape=(frameSize[0],frameSize[1],3))
    inc=keras.applications.DenseNet121(weights="imagenet",include_top=False,  pooling="max",input_shape=(frameSize[0],frameSize[1],3))
    # inc=tf.keras.applications.ResNet152V2(include_top=False, pooling='avg',input_shape=(frameSize[0],frameSize[1],3))
    # inc=tf.keras.applications.DenseNet121(include_top=False, pooling='avg',input_shape=(frameSize[0],frameSize[1],3))
    # inc.fc=layers.Dense(embedDim,activation=tf.nn.gelu)#is this line the issue?
    for layer in inc.layers:
      layer.trainable=False
    x=layers.TimeDistributed(inc)(x)#out is 3D
    # x=layers.TimeDistributed(layers.Conv2D(32,10,activation=tf.nn.gelu))(inputs)
    # x=layers.TimeDistributed(layers.Conv2D(100,10,activation="tanh"))(x)
    # # x=layers.Dense(2,activation="softmax")(x)
    # x=(layers.Conv3D(130,4,activation=tf.nn.gelu))(x)
    # # x=(layers.Conv3D(20,5))(x)
    # # x=(layers.Conv3D(32,10,activation=tf.nn.gelu))(x)
    # # x=(layers.Conv3D(100,5,activation=tf.nn.gelu))(x)
    # print(x.shape)
    # x=layers.TimeDistributed(layers.GlobalMaxPooling2D())(x)
    # x=layers.TimeDistributed(layers.Flatten())(x)
    ############
    # x = layers.Dropout(0.3)(x)
    print(np.shape(x))
    # x=layers.Dense(embedDim,activation=tf.nn.gelu)(x)#is this line the issue?
    # x=layers.Conv1D(embedDim,3,activation=tf.nn.gelu)(x)#is this line the issue?
    print(np.shape(x))
    x = PositionalEmbedding(
        seqLen, x.shape[-1], name="frame_position_embedding"
    )(x)
    # x1=TFPositionalEncoding1D(x.shape[-1])(x)
    # x=layers.Add()([x1,x])
    # x=x1+x
    for _ in range(nTr):
      # x=transformer_encoder(x, embedDim, 32, ff_dim=ff_dim, dropout=trDr)
      # x=TransformerEncoder(embed_dim=embedDim, dense_dim=200, num_heads=numHeads)(x)
      # x=layers.Dropout(0.3)(x)
      x=TransformerEncoder(embed_dim=x.shape[-1], dense_dim=dense_dim2, num_heads=numHeads)(x)
      # x=keras.layers.Attention()([x,x,x])
    print(np.shape(x))
    # res = layers.MultiHeadAttention(
    #       key_dim=200, 
    #       num_heads=8, 
    #       dropout=trDr,
    #       # kernel_regularizer=tf.keras.regularizers.l1(0.02),
    #       # activity_regularizer=tf.keras.regularizers.l1(0.01)
    #   )(x, x)
    # x=layers.Add()([x,res])
    # x = layers.GlobalAveragePooling1D()(x)
    # x=layers.Conv1D(200,2,activation=tf.nn.gelu)(x)#is this line the issue?
    # x=layers.Conv1D(100,2,activation=tf.nn.gelu)(x)#is this line the issue?
    # x = layers.Dropout(0.3)(x)
    # x = layers.Flatten()(x)
    # x = layers.Dense(dense_dim2,activation=tf.nn.gelu)(x)
    # x = layers.Dense(200,activation=tf.nn.gelu)(x)
    # x = layers.GlobalAveragePooling1D()(x)
    x = layers.GlobalMaxPooling1D()(x)
    # print(np.shape(x))
    # x = layers.Dropout(0.3)(x)
    # x = layers.Conv1D(filters=x.shape[-1], kernel_size=1)(x)
    # print(np.shape(x))
    # x=layers.GlobalAveragePooling1D()(x)
    act="softmax" if classes!=2 else "sigmoid"
    # x=layers.Dense(40,activation=tf.nn.gelu)(x)
    x=layers.Dense(classes,activation=act)(x)
    model = keras.Model(inputs, x)
    return model

In [217]:
# data=np.random.rand(2,5,3)+2
# print(data)
# print(np.mean(data))
# print(np.std(data))
# data=layers.LayerNormalization()(data)
# print(data)
# print(np.mean(data))
# print(np.std(data))

In [218]:
# with strategy.scope(): 
with tf.device("/device:GPU:0"): 

  # model=mainModel(embedDim=32*9,numHeads=32,classes=n_classes,dense_dim2=100,trDr=0.5,nTr=8)
  model=mainModel(embedDim=32*11,numHeads=8,classes=n_classes,dense_dim2=4,trDr=0.5,nTr=1)

(None, 45, 128, 128, 3)
(None, 45, 1024)
(None, 45, 1024)
(None, 45, 1024)


In [219]:
%pip install tensorflow_addons
%pip install tensorboard

In [220]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [221]:
from keras.callbacks import TensorBoard
log_dir="temp_log"
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
)

In [222]:
from keras import backend as K
from sklearn.metrics import  precision_score,recall_score,f1_score
from keras.metrics import Recall,Precision
from tensorflow_addons.metrics import F1Score

In [223]:
# with strategy.scope(): 
with tf.device("/device:GPU:0"): 
  class_weight = dict()
  class_weight[0]=1/10
  uniq=len(trainCsv[option].unique())
  for i in range(1,uniq):
    class_weight[i]=(uniq-1)/10
  metrics=['accuracy']
  if option=="label":
    metrics+=[F1Score(n_classes,threshold=0.5),Recall(),Precision()]
  loss="categorical_crossentropy" if option!="label" else "binary_crossentropy"
  # model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.05, nesterov=True),loss='categorical_crossentropy',metrics=['accuracy'])
  # model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001, nesterov=True),loss=loss,metrics=['accuracy',f1_m,precision_m, recall_m])
  model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.1),loss=loss,metrics=metrics)
  # model.compile(optimizer=keras.optimizers.RMSprop(),loss=loss,metrics=metrics)
  # model.compile(optimizer="adam",loss=loss,metrics=metrics)
  train_loss=[]
  val_loss=[]
  train_acc=[]
  val_acc=[]

In [224]:
# # with strategy.scope(): 
# import pickle
# import _pickle as cPickle
# with tf.device("/device:GPU:0"): 
#   model = keras.models.load_model('/content/drive/MyDrive/tf_transformer_4/results2/model')
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/train_loss", "rb") as output:
#     train_loss=cPickle.load(output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/val_loss", "rb") as output:
#     val_loss=cPickle.load(output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/train_acc", "rb") as output:
#     train_acc=cPickle.load(output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/val_acc", "rb") as output:
#     val_acc=cPickle.load(output)
#   trainCsv=pd.read_csv("/content/drive/MyDrive/tf_transformer_4/results2/trainCsv")
#   valCsv=pd.read_csv("/content/drive/MyDrive/tf_transformer_4/results2/valCsv")
#   training_generator = Dataset(trainCsv['Path'].values, trainCsv['classLabel'].values, **params)
#   val_generator = Dataset(valCsv['Path'].values, valCsv['classLabel'].values, **params)

In [225]:
#  with strategy.scope(): 
# # with strategy.scope(): 
#   epochs=20
#   for epoch in range(epochs):
#     start=time.time()
#     print('Epoch: {}/{}'.format(epoch+1,epochs))
#     count=0
#     for i,data in enumerate(training_generator):
#       x,y=data
#       x=x.astype(np.float32)
#       y=y.astype(np.float32)
#       hist=model.fit(x,y,batch_size=batch_size,verbose=0,epochs=1)
#       curr_acc=hist.history['accuracy']
#       curr_loss=hist.history['loss']
#       if count==0:
#         train_loss.append(curr_loss[0])
#         train_acc.append(curr_acc[0])
#       else:
#         train_loss[-1]+=curr_loss[0]
#         train_acc[-1]+=curr_acc[0]
#       count+=1
#       gc.collect()
#     train_loss[-1]/=count
#     train_acc[-1]/=count
#     count=0
#     for i,data in enumerate(val_generator):
#       x,y=data
#       y_pred=model(x)
#       curr_acc=accuracy_score(np.argmax(y,axis=1),np.argmax(y_pred,axis=1))
#       curr_loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False,reduction=tf.keras.losses.Reduction.NONE)(y, y_pred).numpy()
#       curr_loss=np.sum(curr_loss)
#       curr_loss/=batch_size
#       if count==0:
#         val_loss.append(curr_loss)
#         val_acc.append(curr_acc)
#       else:
#         val_loss[-1]+=curr_loss
#         val_acc[-1]+=curr_acc
#       count+=1
#       gc.collect()
#     val_loss[-1]/=count
#     val_acc[-1]/=count
#     end=time.time()
#     gc.collect()
#     print("Time:{} Training loss:{} Training accuracy:{} Validation loss:{} Validation accuracy:{}".format(end-start,train_loss[-1],train_acc[-1],val_loss[-1],val_acc[-1]))  

In [226]:
with tf.device("/device:GPU:0"): 
  class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_batch_end(self, epoch, logs=None):
      # print("Batch end")
      gc.enable()
      gc.collect()
      return
    def on_test_batch_end(self, batch, logs=None):
      # print("Batch end")
      gc.enable()
      gc.collect()
      return
    def on_predict_batch_end(self, batch, logs=None):
      # print("Batch end")
      gc.enable()
      gc.collect()
      return
    def on_train_batch_end(self, batch, logs=None):
      # print("Batch end")
      gc.enable()
      gc.collect()
      return

In [227]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  epochs=8
  start=time.time()
  # divider=1
  # multiplier=2
  # class_weight = dict()
  # class_weight[0]=1 if option=="label" else 1/divider
  # uniq=len(trainCsv[option].unique())
  # print(uniq)
  # for i in range(1,uniq):
  #   class_weight[i]=(uniq-1)/divider if option=="classLabel" else (uniq-1)*multiplier
  # hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, class_weight=class_weight,callbacks=[LossAndErrorPrintingCallback()])
  hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, class_weight=class_count_dict,callbacks=[LossAndErrorPrintingCallback(),tensorboard_callback])
  # hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, callbacks=[LossAndErrorPrintingCallback()])
  end=time.time()
  gc.collect()
  val_loss+=hist.history['val_loss']
  train_loss+=hist.history['loss']
  val_acc+=hist.history['val_accuracy']
  train_acc+=hist.history['accuracy']

Layer PositionalEmbedding has arguments ['self', 'sequence_length', 'output_dim']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/8
104/104 [==============================] - 950s 9s/step - loss: 43.6562 - accuracy: 0.3005 - val_loss: 8.1425 - val_accuracy: 0.2985
Epoch 2/8
104/104 [==============================] - 875s 8s/step - loss: 5.4142 - accuracy: 0.5243 - val_loss: 5.8518 - val_accuracy: 0.3233
Epoch 3/8
104/104 [==============================] - 885s 8s/step - loss: 2.7515 - accuracy: 0.6247 - val_loss: 5.2667 - val_accuracy: 0.3685
Epoch 4/8
104/104 [==============================] - 888s 9s/step - loss: 1.9386 - accuracy: 0.6767 - val_loss: 3.5857 - val_accuracy: 0.4224
Epoch 5/8
104/104 [==============================] - 890s 9s/step - loss: 1.3925 - accuracy: 0.7326 - val_loss: 2.8572 - val_accuracy: 0.4892
Epoch 6/8
104/104 [==============================] - 859s 8s/step - loss: 1.2673 - accuracy: 0.7536 - val_loss: 2.1606 - val_accuracy: 0.5409
Epoch 7/8
104/104 [==============================] - 854s 8s/step - loss: 1.0499 - accuracy: 0.7837 - val_loss: 1.6635 - val_accuracy: 0.5873
Epoch

In [ ]:
with tf.device("/device:GPU:0"): 
# with strategy.scope(): 
  epochs=8
  start=time.time()
  # divider=1
  # multiplier=2
  # class_weight = dict()
  # class_weight[0]=1 if option=="label" else 1/divider
  # uniq=len(trainCsv[option].unique())
  # print(uniq)
  # for i in range(1,uniq):
  #   class_weight[i]=(uniq-1)/divider if option=="classLabel" else (uniq-1)*multiplier
  # hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, class_weight=class_weight,callbacks=[LossAndErrorPrintingCallback()])
  hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, class_weight=class_count_dict,callbacks=[LossAndErrorPrintingCallback(),tensorboard_callback])
  # hist=model.fit(x=training_generator,validation_data=val_generator,epochs=epochs,verbose=1, callbacks=[LossAndErrorPrintingCallback()])
  end=time.time()
  gc.collect()
  val_loss+=hist.history['val_loss']
  train_loss+=hist.history['loss']
  val_acc+=hist.history['val_accuracy']
  train_acc+=hist.history['accuracy']

Layer PositionalEmbedding has arguments ['self', 'sequence_length', 'output_dim']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/8
104/104 [==============================] - 957s 9s/step - loss: 0.8120 - accuracy: 0.8182 - val_loss: 1.3688 - val_accuracy: 0.6401
Epoch 2/8
104/104 [==============================] - 931s 9s/step - loss: 0.7822 - accuracy: 0.8398 - val_loss: 1.1434 - val_accuracy: 0.6864
Epoch 3/8
104/104 [==============================] - 929s 9s/step - loss: 0.6756 - accuracy: 0.8462 - val_loss: 1.3166 - val_accuracy: 0.6638
Epoch 4/8
104/104 [==============================] - 926s 9s/step - loss: 0.6026 - accuracy: 0.8618 - val_loss: 1.0950 - val_accuracy: 0.7037
Epoch 5/8
104/104 [==============================] - 929s 9s/step - loss: 0.5155 - accuracy: 0.8819 - val_loss: 1.0555 - val_accuracy: 0.7166
Epoch 6/8
104/104 [==============================] - 926s 9s/step - loss: 0.4774 - accuracy: 0.8924 - val_loss: 1.1440 - val_accuracy: 0.7069
Epoch 7/8
104/104 [==============================] - 938s 9s/step - loss: 0.4381 - accuracy: 0.9011 - val_loss: 1.1366 - val_accuracy: 0.7155
Epoch 

In [ ]:
#this seems fast 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
tensorboard --logdir temp_log

In [ ]:
# import pickle
# with tf.device("/device:GPU:0"):
#   model.save('/content/drive/MyDrive/tf_transformer_5/results/model')
#   with open("/content/drive/MyDrive/tf_transformer_5/results/train_loss", "wb") as output:
#     pickle.dump(train_loss,output)
#   with open("/content/drive/MyDrive/tf_transformer_5/results/val_loss", "wb") as output:
#     pickle.dump(val_loss,output)
#   with open("/content/drive/MyDrive/tf_transformer_5/results/train_acc", "wb") as output:
#     pickle.dump(train_acc,output)
#   with open("/content/drive/MyDrive/tf_transformer_5/results/val_acc", "wb") as output:
#     pickle.dump(val_acc,output)


In [ ]:
# with tf.device("/device:GPU:0"):
#   trainCsv.to_csv("/content/drive/MyDrive/tf_transformer_5/results/trainCsv")
#   testCsv.to_csv("/content/drive/MyDrive/tf_transformer_5/results/testCsv")
#   valCsv.to_csv("/content/drive/MyDrive/tf_transformer_5/results/valCsv")

In [ ]:
# import pickle
# with tf.device("/device:GPU:0"):
#   model.save('/content/drive/MyDrive/tf_transformer_4/results2/model')
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/train_loss", "wb") as output:
#     pickle.dump(train_loss,output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/val_loss", "wb") as output:
#     pickle.dump(val_loss,output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/train_acc", "wb") as output:
#     pickle.dump(train_acc,output)
#   with open("/content/drive/MyDrive/tf_transformer_4/results2/val_acc", "wb") as output:
#     pickle.dump(val_acc,output)
#   trainCsv.to_csv("/content/drive/MyDrive/tf_transformer_4/results2/trainCsv")
#   valCsv.to_csv("/content/drive/MyDrive/tf_transformer_4/results2/valCsv")

In [ ]:
# # for data in val_generator:
# for data in training_generator:
#   print(np.count_nonzero(data[0]))
#   # break 

In [ ]:
# import os
# os.listdir("/content/drive/MyDrive/dcsass/DCSASS Dataset")

In [ ]:
# with tf.device("/device:GPU:0"):
#   # del model
#   gc.collect()